# First steps - Coursera Data Analysis
---
Import necessary libraries for data analysis and plotting.

In [1]:
#%pip install -r requirements.txt

import pandas as pd
import langdetect as ld
from IPython.display import Markdown as md
from helper import convert_str_to_num, scatter_plotly, correl_heat_map, compare_lang, bar_plot, plot_lan, capitalize_first_word, course_count_rating_plot, perform_anova_and_visualize

---

>
> Source: https://www.kaggle.com/datasets/siddharthm1698/coursera-course-dataset
>
> This dataset contains 6 columns of features on 891 course records. 
> 
> The detailed description:
>
> - course_title : Contains the course title.
> - course_organization : It tells which organization is conducting the courses.
> - course_Certificate_type : It has details about what are the different certifications available in courses.
> - course_rating : It has the ratings associated with each course.
> - course_difficulty : It tells about how difficult or what is the level of the course.
> - course_students_enrolled : It has the number of students that are enrolled in the course.
>
> Credit: https://github.com/Siddharth1698
>
>

---

Read dataset from csv file located in the working directory, and set the included numeric identifier column as index for the DataFrame.

In [2]:
data = pd.read_csv("coursea_data.csv", index_col=0)

---

Have a rough look at the dataset.

In [3]:
data.sort_index(inplace=True)
data.head()

,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled
0,IBM Data Science,IBM,PROFESSIONAL CERTIFICATE,4.6,Beginner,480k
1,Introduction to Data Science,IBM,SPECIALIZATION,4.6,Beginner,310k
2,The Science of Well-Being,Yale University,COURSE,4.9,Mixed,2.5m
3,Python for Everybody,University of Michigan,SPECIALIZATION,4.8,Beginner,1.5m
4,Google IT Support,Google,PROFESSIONAL CERTIFICATE,4.8,Beginner,350k


---

## Data cleaning

### Check for missing values.

In [4]:
data.isna().any(axis = None)

False

None of the Data in the DataFrame is null.

I decided not to drop any of the cells in case of missing values, because this is a relatively limited dataset.

If there were empty cells in the DataFrame:

In [5]:
data.fillna("N/A", inplace=True)

---

### Check for duplicates in the DataFrame.

In [6]:
data.duplicated().any(axis=None)

False

No duplicate in the data. 

If there were any duplicates:

In [7]:
data.drop_duplicates(inplace=True)

---

## Data wrangling


Unfortunately the number of enrolled students is given as: 5.3k, which has to be converted to a numeric value to gain statistical insights.

In [8]:
if data["course_students_enrolled"].dtype != int: #added this check to avoid error on repeated running
    data["course_students_enrolled"] = data["course_students_enrolled"].map(convert_str_to_num)

data.head()

,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled
0,IBM Data Science,IBM,PROFESSIONAL CERTIFICATE,4.6,Beginner,480000
1,Introduction to Data Science,IBM,SPECIALIZATION,4.6,Beginner,310000
2,The Science of Well-Being,Yale University,COURSE,4.9,Mixed,2500000
3,Python for Everybody,University of Michigan,SPECIALIZATION,4.8,Beginner,1500000
4,Google IT Support,Google,PROFESSIONAL CERTIFICATE,4.8,Beginner,350000


---

There are courses in multiple languages, so I will add a feature to track the language of the course.

In [9]:
if not 'language' in data.columns:
    data['language'] = data['course_title'].apply(ld.detect)


The language detection was not reliable enough, so I proofed the 170 courses that weren't detected as English, and manually set them to English if needed.

In [10]:
lang_false_idx = [0,3,4,5,11,13,24,28,33,34,41,43,46,48,75,76,78,80,92,101,102,104,110,115,119,129,137,149,153,160,167,180,188,206,210,220,239,241,256,262,268,283,285,286,304,309,317,325,326,342,365,381,390,391,410,434,441,446,455,458,487,489,490,501,519,537,539,546,553,558,578,592,595,601,611,614,639,662,682,683,693,697,706,731,732,757,761,762,817,839,855,886,887]

for id in lang_false_idx:
    data.at[id,'language'] = 'en'
    
lang_df = data[data['language']!='en'].loc[:, ['course_title', 'language']]

In [11]:
plot_lan(lang_df)

In [12]:
records = data.index.size
en_count = data[data.language =='en'].index.size
es_count = data[data.language =='es'].index.size
fr_count = data[data.language =='fr'].index.size
ru_count = data[data.language =='ru'].index.size
pt_count = data[data.language =='pt'].index.size

lang_raw = [
    {'language': 'English', 'worldwide': 18.8, 'in dataset': round(en_count/records*100, 1)},
    {'language': 'Spanish', 'worldwide': 6.9, 'in dataset': round(es_count/records*100, 1)},
    {'language': 'French', 'worldwide': 3.4, 'in dataset': round(fr_count/records*100, 1)},
    {'language': 'Russian', 'worldwide': 3.2, 'in dataset': round(ru_count/records*100, 1)},
    {'language': 'Portugese', 'worldwide': 3.2, 'in dataset': round(pt_count/records*100, 1)},
    ]

#Source: https://en.wikipedia.org/wiki/List_of_languages_by_total_number_of_speakers

lang_comparison = pd.DataFrame(lang_raw)
lang_comparison


,language,worldwide,in dataset
0,English,18.8,90.2
1,Spanish,6.9,4.9
2,French,3.4,1.1
3,Russian,3.2,1.6
4,Portugese,3.2,0.7


In [13]:
compare_lang(lang_comparison)

> Based on this comparison, it seems practical to develop courses for English and Spanish speakers.

---

# Exploratory data analysis

Filtering for institutions that have more than x courses. 

From a business perspective it might be smart to target these institutions with our services, because they are more likely to have a budget to be able to develop their materials based on analytics provided by us.

In the color coding the respective ratings of the institution reviews are also included.

Modify x below as desired.

In [39]:
x = 20

course_count_rating_plot(data, x)


---

First I will display the basic statistical characteristics of both categorical and numerical features.

Filtering the numeric features into a new DataFrame, for straight-forward handling. 


In [15]:
num_cols = data.select_dtypes('number').columns
numeric_data = data.loc[:, num_cols]

I will take a look at the numeric statistical characteristics.

In [16]:
num_insights = numeric_data.describe().T
num_insights.index = num_insights.index.map(capitalize_first_word)
num_insights

,count,mean,std,min,25%,50%,75%,max
Course rating,891.0,4.677329,0.162225,3.3,4.6,4.7,4.8,5.0
Course students enrolled,891.0,90552.076319,181936.453852,1500.0,17500.0,42000.0,99500.0,3200000.0


The average rating is relatively high at 4.68, so the site seems to accomodate quality content based on user reviews.

The minimum value is extremely low, it might be a good idea to take a look into it.

The number of enrolled students averages to 90k, but the max is comperatively much higher at 3.2m, it would be interesting to figure out why that course is so popular.

---

Checking the corresponding basic statistical characteristics for categorical features.

In [17]:
cat_cols=data.columns.drop(num_cols).drop(['course_title', 'course_organization'])
cat_insights = data[cat_cols].describe().T
cat_insights.index = cat_insights.index.map(capitalize_first_word)
cat_insights

,count,unique,top,freq
Course certificate type,891,3,COURSE,582
Course difficulty,891,4,Beginner,487
Language,891,14,en,804


Based on this dataset the audience at coursera is mostly interested in courses aimed at beginners, as rougly 500 of the 891 records represent. 

Visualization of the categorical features and their distributions to support the statement above.

In [18]:
bar_plot(data, cat_cols)


---

I will continue with the thorough analysis of the numerical features.

### Outlier detection

> Calculating the limit for each feature and checking if the difference of the values from the mean is within the tolerance of 3 standard deviations.
>
> ![alt text](norm_dist.jpg)
>
> Source: Adobe Stock

The result is a bool DataFrame of the same structure as the filtered numerical data.

In [19]:
lim = abs((numeric_data - numeric_data.mean())/numeric_data.std(ddof = 0)) <3

Filtering the dataset to check if the row has any outliers.

If it does, it gets indicated in the bool Series called filter, which then filters the whole, original dataset to retrieve the records that have outliers.

In [20]:
filter = lim.all(axis = 1)
outlier_records = data[~filter]

If we want to check which values are outliers, we can use the lim bool DataFrame to filter.

In [21]:
outlier_items = numeric_data[~lim]
outlier_items = outlier_items.dropna(how='all').dropna(axis = 1, how='all')

After filtering the outlier items I am creating a new DataFrame to display them.

In [22]:
outlier_stack = outlier_items.stack().reset_index().set_index("level_1")
outlier_df=pd.DataFrame(outlier_stack).rename(columns={"level_0": "at_index", 0: "value"}).rename_axis("feature")

Creating scatter plots for visualization of the values, the outliers and the tolerance zone for each feature that has outliers.

In [23]:
columns_to_keep = ~lim.all()

to_plot = data[num_cols].loc[:, columns_to_keep]
        
for feature in to_plot:
    fig = scatter_plotly(dataframe=to_plot, x= to_plot.index, y = feature, outlier=outlier_df, feature=feature)
    #manual modifications based on the features' nature
    if feature == 'course_rating':
        fig.add_hline(y=5.0, line_color ='violet', line_width =2, line_dash= 'dot', annotation_text = 'Max valid value: 5', annotation_position = 'right')
        fig.show()
    elif feature == 'course_students_enrolled':
        fig.update_layout(coloraxis_colorscale = 'Tropic')
        fig.add_hline(y=0.0, line_color ='violet', line_width =2, line_dash= 'dot', annotation_text = 'Min valid value: 0', annotation_position = 'right')    
        fig.show()


The outliers here can all be further examined to find the reason for their relative extremity.

It could also be an explanation, that the data set is relatively small, and in case of a larger data set they might not even be outliers.

---

The following records are outliers in one of their features. 

I will try to find the reason for the extreme values. 

Displaying the records with outlier values in the feature 'course_rating' in an ascending order.

In [24]:
data.iloc[outlier_df[outlier_df.index == 'course_rating'].at_index].sort_values(by='course_rating')

,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled,language
767,How To Create a Website in a Weekend! (Project...,The State University of New York,COURSE,3.3,Mixed,140000,en
352,Machine Learning and Reinforcement Learning in...,New York University,SPECIALIZATION,3.7,Intermediate,29000,en
178,Machine Learning for Trading,Google Cloud,SPECIALIZATION,3.9,Intermediate,15000,en
779,iOS App Development with Swift,University of Toronto,SPECIALIZATION,3.9,Intermediate,76000,en
659,"Introduction to Trading, Machine Learning & GCP",New York Institute of Finance,COURSE,4.0,Intermediate,13000,en
873,Mathematics for Machine Learning: PCA,Imperial College London,COURSE,4.0,Intermediate,33000,en
609,How to Start Your Own Business,Michigan State University,SPECIALIZATION,4.1,Beginner,34000,en


The following courses have a rating below 4.0, their organizers might be open to feedback to increase their user ratings.

In [25]:
data[data.course_rating<4].sort_values(by = 'course_rating')
poorly_rated_orgs = '\n>- '.join(data[data.course_rating<4].course_organization.values)
md('>**TO DO:**\n>\n>Reach out to:\n>- {} \n>\n> to offer an analysis-based plan to increase user ratings.'.format(poorly_rated_orgs))

>**TO DO:**
>
>Reach out to:
>- Google Cloud
>- New York University
>- The State University of New York
>- University of Toronto 
>
> to offer an analysis-based plan to increase user ratings.

Displaying the records with outlier values in the feature 'course_students_enrolled' in a descending order.

In [26]:
data.iloc[outlier_df[outlier_df.index == 'course_students_enrolled'].at_index].sort_values(by='course_students_enrolled', ascending = False)

,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled,language
6,Machine Learning,Stanford University,COURSE,4.9,Mixed,3200000,en
2,The Science of Well-Being,Yale University,COURSE,4.9,Mixed,2500000,en
3,Python for Everybody,University of Michigan,SPECIALIZATION,4.8,Beginner,1500000,en
47,Programming for Everybody (Getting Started wit...,University of Michigan,COURSE,4.8,Mixed,1300000,en
13,Data Science,Johns Hopkins University,SPECIALIZATION,4.5,Beginner,830000,en
44,Career Success,"University of California, Irvine",SPECIALIZATION,4.4,Beginner,790000,en
175,English for Career Development,University of Pennsylvania,COURSE,4.8,Mixed,760000,en
40,Successful Negotiation: Essential Strategies a...,University of Michigan,COURSE,4.8,Mixed,750000,en
15,Data Science: Foundations using R,Johns Hopkins University,SPECIALIZATION,4.6,Beginner,740000,en
5,Deep Learning,deeplearning.ai,SPECIALIZATION,4.8,Intermediate,690000,en


The following courses have students enrolled above 1m, so further data collection and anaylisis can provide insights on how to enroll the most students.

Also, by looking at the titles we can see that various fields of IT is are the *hot topic* on coursera. 

In [27]:
data[data.course_students_enrolled>1000000].sort_values(by = 'course_students_enrolled', ascending=False)


,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled,language
6,Machine Learning,Stanford University,COURSE,4.9,Mixed,3200000,en
2,The Science of Well-Being,Yale University,COURSE,4.9,Mixed,2500000,en
3,Python for Everybody,University of Michigan,SPECIALIZATION,4.8,Beginner,1500000,en
47,Programming for Everybody (Getting Started wit...,University of Michigan,COURSE,4.8,Mixed,1300000,en


In [28]:
top_enrollment_courses = '\n'.join(
    f">- #{index} {title}"
    for index, title in data[data.course_students_enrolled > 1000000]['course_title'].items()
)
md('>**TO DO:**\n>\n>Collect more data on:\n>\n{}'.format(top_enrollment_courses))

>**TO DO:**
>
>Collect more data on:
>
>- #2 The Science of Well-Being
>- #3 Python for Everybody
>- #6 Machine Learning
>- #47 Programming for Everybody (Getting Started with Python)

There is an option to create a correlation heatmap to check if there is a correlation between the number of enrolled students and the rating, but in this case with a single relevant value, a single output should be more practical.

> Statistical correlation measures the strength and direction of the relationship between two variables.

In [29]:
num_corr_mtx = data.loc[:, ['course_rating', 'course_students_enrolled']].corr()

#correl_heat_map(num_corr_mtx)

print(f'The correlation coefficient between the number of enrolled students and the ratings: {num_corr_mtx.iloc[0,1]:.2f}, which is statistically insignificant.' )


The correlation coefficient between the number of enrolled students and the ratings: 0.07, which is statistically insignificant.


Aggregating the numerical values based on institution.

In [30]:
data.groupby('course_organization')['course_rating'].describe().sort_values(by = 'count', ascending = False).head()

,count,mean,std,min,25%,50%,75%,max
course_organization,,,,,,,,
University of Pennsylvania,59.0,4.696610,0.137673,4.2,4.600,4.7,4.8,4.9
University of Michigan,41.0,4.729268,0.095509,4.5,4.700,4.8,4.8,4.9
Google Cloud,34.0,4.582353,0.162314,3.9,4.500,4.6,4.7,4.8
Duke University,28.0,4.664286,0.147106,4.2,4.600,4.7,4.8,4.9
Johns Hopkins University,28.0,4.660714,0.157149,4.3,4.575,4.7,4.8,4.9


Aggregating the numerical values based on language.

In [31]:
lang_students = data.groupby('language')['course_students_enrolled'].aggregate('sum')
lang_students


language
bg       21000
ca       65000
da       58000
de      223000
en    75481700
es     3175400
fr      365100
hu       46000
it      200000
nl       66000
no       57000
pt      278500
ro       73400
ru      571800
Name: course_students_enrolled, dtype: int64

--- 

**Checking if there is a correlation between numerical and categorical features with ANOVA (Analysis of Variance).**

In this case, we are testing if there is a difference between the average course ratings and the number of enrolled students between different organizations, types of certificates, and difficulty levels.

> **The result of the ANOVA is: p-value and F-value**
>
> If the p-value is below 0.05, the difference between group means is considered statistically significant and unlikely to be due to chance.
>
> A higher F-value suggests a more significant difference between group means relative to the variability within groups.

*Does the average course rating differ significantly between different organizations (course_organization)?*

-> Yes, as the p-value is lower than 0.05, the organization has a non-coincidental effect on the average course rating.


In [32]:
perform_anova_and_visualize(data, categorical_feature="course_organization", numerical_feature="course_rating")

*Does the average course rating differ significantly between different types of output certificates (course_Certificate_type)?*

-> Yes, as the p-value is lower than 0.05, the type of output certificate has a non-coincidental effect on the average course rating, but we have very little amount of data to base the statement on.

In [33]:
perform_anova_and_visualize(data, categorical_feature="course_Certificate_type", numerical_feature="course_rating")

*Does the average course rating differ significantly between different types of output certificates (course_Certificate_type)?*

-> Yes, the course difficulty has an effect on the average course rating, statistically the deviations are considered non-accidental.

In [34]:
perform_anova_and_visualize(data, categorical_feature="course_difficulty", numerical_feature="course_rating")

*Does the number of enrolled students differ significantly between course difficulties?*

-> No, statistically speaking the difficulty has no direct effect on the number of enrolled students.

In [35]:
perform_anova_and_visualize(data, categorical_feature="course_difficulty", numerical_feature="course_students_enrolled")

*Does the number of enrolled students differ significantly between different course organizations?*

-> No, statistically speaking the organization has no direct effect on the number of enrolled students.

In [36]:
perform_anova_and_visualize(data, categorical_feature="course_organization", numerical_feature="course_students_enrolled")

*Does the number of enrolled students differ significantly between different course certificates?*

-> Yes, statistically speaking the certificate type has an effect on the number of enrolled students.

In [37]:
perform_anova_and_visualize(data, categorical_feature="course_Certificate_type", numerical_feature="course_students_enrolled")

---

## Conclusion

The measures listed as TO-DOs could improve the performance of the relevant courses and possibly initiate cooperation with the organizations to gain further access to valuable information.

The result of my analysis suggests that courses providing a professional certificate on a beginner level are the most popular, and likely to have a good rating.

It is most practical to develop content for English and Spanish-speaking learners.

---

I would gather more data to support my theories, and also to gain deeper insights.

I would find a solution to determine the language with more confidence, and also apply tags to records based on topic. Unfortunately my solution was not satisfactory for this problem, so I did not include it in the submitted version.

Based on the topic there could be an extra layer in the analysis that is not directly included in the original dataset.

Also I would gather data on the course being completed, the time invested in completion, and the ratios of completion vs. enrollment, pricing, scholarships.

I would also include better business insights and suggestions after thorough research.